In [1]:
import pandas as pd
import os
import ast
import numpy as np
from Bio.SeqUtils.ProtParam import ProteinAnalysis
pd.options.display.max_colwidth = 100
pd.options.display.float_format = '{:.2f}'.format

In [2]:
all_results = []
for f in os.listdir('.'):
    if '.data' in f:
        with open(f, 'r') as g:
            all_results.append(ast.literal_eval(g.read()))

In [3]:
def standardize_to_uM(concentration, unit, sequence):
    concentration = concentration.replace(' ', '')
    try:
        concentration = float(concentration)
    except:
        return None
    if unit == 'uM' or unit == u'\xb5M' or unit == u'uM)':
        return concentration
    elif unit == 'ug/ml' or unit == u'\xb5g/ml' or unit == u'ug/ml)':
        try:
            molWt = ProteinAnalysis(sequence).molecular_weight()
        except ValueError:
            return None
        return concentration * 1000/molWt
    elif unit == 'nmol/g' or unit == 'pmol/mg':
        #1g, at density of 1g/mL, is 1mL, so nmol/g is nmol/mL = umol/L = uM yay!
        return concentration
    else:
        # print 'Unit not recognized: ' + unit
        return None

In [6]:
def convert_result_to_rows(sequence, result):
    rows = []
    if 'bacteria' not in result:
        return rows
    for bacterium, strain in result['bacteria']:
        
        rows.append({
            'bacterium': bacterium,
            'strain': strain,
            'sequence': sequence.upper(),
            'url_source': result['url_sources'][0],
            'value': standardize_to_uM(
                result['bacteria'][(bacterium, strain)]['value'],
                result['bacteria'][(bacterium, strain)]['unit'],
                sequence
            ),
            'modifications': result['modifications'] if 'modifications' in result else [],
            'unit': 'uM'
        })
        if rows[-1]['value']:
            rows[-1]['value'] = np.log10(rows[-1]['value'])
    return rows

In [8]:
rows = []
for result_set in all_results:
    for sequence in result_set:
        for row in convert_result_to_rows(sequence, result_set[sequence]):
            rows.append(row)

In [9]:
df = pd.DataFrame(rows)

In [38]:
def is_modified(modifications_list):
    return len(modifications_list) > 0

df['is_modified'] = df.modifications.apply(is_modified)

In [39]:
def has_non_cterminal_modification(modifications_list):
    return any(['C-Term' not in modification for modification in modifications_list])

df['has_non_cterminal_modification'] = df.modifications.apply(has_non_cterminal_modification)
df['has_cterminal_modification'] = df.is_modified & ~df.has_non_cterminal_modification

In [40]:
# Clean sequences
df.sequence = df.sequence.str.strip()
df = df.loc[df.sequence != '/']

In [41]:
# Exclude sequences with modifications

# Exclude rows from YADAMP and CAMP for having no modification data

#     Unless that sequence is in another DB

In [42]:
df = df.loc[df.has_non_cterminal_modification == False]

no_modification_data_sources = ['camp3', 'yadamp']

def datasource_has_modifications(cell):
    # Everything except CAMP and YADAMP has modification data
    return not any([s in cell for s in no_modification_data_sources])

df['_datasource_has_modifications'] = df['url_source'].apply(datasource_has_modifications)

sequences_containing_modifications = set(df.loc[df._datasource_has_modifications == True, 'sequence'])
def sequence_has_modification_data(cell):
    # If the sequence is labeled modifictationless in another database it's OK
    return cell in sequences_containing_modifications

df['_sequence_has_modifications'] = df['sequence'].apply(sequence_has_modification_data)

df['modification_verified'] = df['_sequence_has_modifications'] | df['_datasource_has_modifications']

df = df.loc[df.modification_verified == True]

In [16]:
staph = df.loc[df.bacterium.str.contains('S. aureus')].groupby('sequence')['value'].mean().dropna()
ecoli = df.loc[df.bacterium.str.contains('E. coli')].groupby('sequence')['value'].mean().dropna()
pseudomonas = df.loc[df.bacterium.str.contains('P. aeruginosa')].groupby('sequence')['value'].mean().dropna()
streptococcus = df.loc[df.bacterium.str.contains('S. mutans')].groupby('sequence')['value'].mean().dropna()
bacillus = df.loc[df.bacterium.str.contains('B. subtilis')].groupby('sequence')['value'].mean().dropna()

In [20]:
many_bacteria = pd.concat([ecoli, pseudomonas, streptococcus, staph, bacillus], axis=1).reset_index()
many_bacteria.columns = ['index', 'ecoli', 'pseudomonas', 'streptococcus', 'staph', 'bacillus']
many_bacteria.corr()

,ecoli,pseudomonas,streptococcus,staph,bacillus
ecoli,1.00,0.78,0.71,0.67,0.69
pseudomonas,0.78,1.00,0.52,0.65,0.60
streptococcus,0.71,0.52,1.00,0.83,0.82
staph,0.67,0.65,0.83,1.00,0.67
bacillus,0.69,0.60,0.82,0.67,1.00


In [43]:
ecoli_df = df.loc[df.bacterium.str.contains('E. coli')]

In [29]:
character_dict = set([character for sequence in ecoli_df.sequence for character in sequence])
max_sequence_length = int(ecoli_df.sequence.str.len().describe(percentiles=[0.95])['95%'])
character_to_index = {
    character: i
    for i, character in enumerate(character_dict)
}

def sequence_to_vector(sequence):
    default = np.zeros([max_sequence_length, len(character_to_index)])
    for i, character in enumerate(sequence[:max_sequence_length]):
        default[i][character_to_index[character]] = 1
    return default

In [ ]:
features = {}

In [33]:
labels = np.array(ecoli_df.value)

sequence
AAAAAAAAAAGIGKFLHSAKKFGKAFVGEIMNS                                            2.10
AAAAAAAIKMLMDLVNERIMALNKKAKK                                                 1.00
AAAKAALNAVLVGANA                                                             1.90
AACSDRAHGHICESFKSFCKDSGRNGVKLRANCKKTCGLC                                     0.25
AAGMGFFGAR                                                                   1.11
AAGRYQLLSRYWDAYR                                                             2.18
AAHCIALRKGYK                                                                 1.48
AAHCIQLGKR                                                                   1.48
AAHCIVLHHN                                                                   1.48
AAHCLAIGRK                                                                   0.88
AAHCLAIGRR                                                                   1.13
AAHCLAKRKK                                                                   1.48
AAHCLAM

In [ ]:
# Need to make df.has_cterminal_modification a feature